In [1]:
import model_utils.utils_model as us1
from model_utils.util_s1_long_seq_bb_method import predict_long_seq_wrapper
import numpy as np
import pandas as pd

In [2]:
def check_joined_df(df1, df2):
    
    def check_arr_eq(arr1, arr2):
        arr1 = np.sort(np.asarray(arr1))
        arr2 = np.sort(np.asarray(arr2))
        np.testing.assert_array_almost_equal(arr1, arr2)
    
    df_check = pd.merge(df1, pd.DataFrame(df2), on=['bb_x', 'bb_y', 'siz_x', 'siz_y'], how='outer')
    assert len(df1) == len(df2)
    assert len(df1) == len(df_check)
    
    for _, row in df_check.iterrows():
        if 'prob_sm_x' in row.keys() and 'prob_sm_y' in row.keys():
            check_arr_eq(row['prob_sm_x'], row['prob_sm_y'])
        if 'prob_sl_x' in row.keys() and 'prob_sl_y' in row.keys():   
            check_arr_eq(row['prob_sl_x'], row['prob_sl_y'])

In [3]:
predictor_s1 = us1.Predictor('v1.0')

Loading know version v1.0 with params {'num_filters': [32, 32, 64, 64, 64, 128, 128], 'filter_width': [9, 9, 9, 9, 9, 9, 9], 'dropout': 0.0}


In [4]:
# # len 200
# seq = 'ACGATGACGATAGACGCGACGACAGCGATGACGATGACGATAGACGCGACGACAGCGATGATGGCGATAAAATTAGGCGCTAGCCCGGATGGAGGGGGAGGAGGAACCCCCTCTGATGCTGATGATCGATATGGCGATAAAATTAGGCGCTAGCCCGGATGGAGGGGGAGGAGGAACCCCCTCTGATGCTGATGATCGAT'


# # len 56
# seq = 'ACGATGACGATAGACGCGACGACAGCGATGACGATGACGATAGACGACGACAGCGA'

# # len 80
# seq = 'ACGATGACGATAGACGCGTATTAGACGAGACGGACGTAGACGACGACAGCGATGACGATGACGATAGACGACGACAGCGA'

# # len 60
# seq = 'ACGATGACGATAGAAGCACGCGACGACAGCGATGACGATGACGATAGACGACGACAGCGA'

# len 112
seq = 'ACGATGACGATAGACGCGACGACACGATGACGATAGACGCGACGACAGCGATGACGATGACGATAGACGACGACAGCGAAGCGATGACGATGACGATAGACGACGACAGCGA'

In [5]:
print(len(seq))

112


In [6]:
# non-split (original interface)
stem_1, iloop_1, hloop_1 = predictor_s1.predict_bb(seq, threshold=0.1, topk=1, perc_cutoff=0)

/Users/alicegao/work/psi-lab-sandbox/meetings/2021_02_02/model_utils/utils_model.py:952: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yp = self.model(torch.tensor(de.x_torch))


In [7]:
# split (long sequence interface, array method)
# stem_2, iloop_2, hloop_2 = predictor_s1.predict_bb_split(seq, threshold=0.1, topk=1, perc_cutoff=0, 
#                                                          patch_size=28, trim_size=28)

# stem_2, iloop_2, hloop_2 = predictor_s1.predict_bb_split(seq, threshold=0.1, topk=1, perc_cutoff=0, 
#                                                          patch_size=40, trim_size=28)


# stem_2, iloop_2, hloop_2 = predictor_s1.predict_bb_split(seq, threshold=0.1, topk=1, perc_cutoff=0, 
#                                                          patch_size=80, trim_size=28)

# stem_2, iloop_2, hloop_2 = predictor_s1.predict_bb_split(seq, threshold=0.1, topk=1, perc_cutoff=0, 
#                                                          patch_size=30)

stem_2, iloop_2, hloop_2 = predictor_s1.predict_bb_split(seq, threshold=0.1, topk=1, perc_cutoff=0, 
                                                         patch_size=100)


Input region: 0-112, 0-112
Output region: 0-100, 0-100


/Users/alicegao/work/psi-lab-sandbox/meetings/2021_02_02/model_utils/utils_model.py:837: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yp = self.model(torch.tensor(de.x_torch))


Selecting array output range: 0-100, 0-100
Input region: 0-112, 72-112
Output region: 0-100, 100-112
Selecting array output range: 0-100, 28-40
Input region: 72-112, 0-112
Output region: 100-112, 0-100
Selecting array output range: 28-40, 0-100
Input region: 72-112, 72-112
Output region: 100-112, 100-112
Selecting array output range: 28-40, 28-40


In [8]:
# not working for now, see README

# # split (long sequence interface, bounding box method)
# stem_3, iloop_3, hloop_3 = predict_long_seq_wrapper(seq, patch_size=100, predictor_s1=predictor_s1, 
#                                                     threshold=0.1, topk=1, perc_cutoff=0)

In [9]:
# print(len(stem_1), len(stem_2), len(stem_3))
print(len(stem_1), len(stem_2))

33 33


In [10]:
# print(len(iloop_1), len(iloop_2), len(iloop_3))
print(len(iloop_1), len(iloop_2))

99 99


In [11]:
# print(len(hloop_1), len(hloop_2), len(hloop_3))
print(len(hloop_1), len(hloop_2))

12 12


In [12]:
check_joined_df(pd.DataFrame(stem_1), pd.DataFrame(stem_2))
# check_joined_df(pd.DataFrame(stem_1), pd.DataFrame(stem_3))

In [13]:
check_joined_df(pd.DataFrame(iloop_1), pd.DataFrame(iloop_2))
# check_joined_df(pd.DataFrame(iloop_1), pd.DataFrame(iloop_3))

In [14]:
check_joined_df(pd.DataFrame(hloop_1), pd.DataFrame(hloop_2))
# check_joined_df(pd.DataFrame(hloop_1), pd.DataFrame(hloop_3))

In [15]:
import torch

In [16]:
# de = us1.SeqPairEncoder(seq_1, seq_2)
de = us1.DataEncoder('ACGT')
yp = predictor_s1.model(torch.tensor(de.x_torch))

/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
yp['stem_on'][0, 0, :, :]

tensor([[1.6912e-04, 9.7935e-05, 1.0809e-03, 2.5290e-02],
        [9.7002e-05, 1.0694e-04, 5.3612e-03, 1.3337e-02],
        [6.5571e-04, 2.1104e-03, 2.2309e-02, 6.3060e-02],
        [3.2504e-03, 2.5612e-03, 1.9993e-02, 1.4175e-02]],
       grad_fn=<SliceBackward>)

In [18]:
de = us1.DataEncoder('NNACGT')
yp2 = predictor_s1.model(torch.tensor(de.x_torch))

/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [19]:
yp2['stem_on'][0, 0, 2:, 2:]

tensor([[0.0001, 0.0013, 0.0010, 0.0088],
        [0.0009, 0.0027, 0.0013, 0.0021],
        [0.0004, 0.0033, 0.0012, 0.0026],
        [0.0030, 0.0008, 0.0019, 0.0020]], grad_fn=<SliceBackward>)

In [20]:
de = us1.SeqPairEncoder('ACGT', 'AAACGT')
# de = us1.DataEncoder('ACGT')
yp = predictor_s1.model(torch.tensor(de.x_torch))

/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
